In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# from transformers import BertTokenizer, BertForSequenceClassification, AdamW
# from torch.utils.data import DataLoader, TensorDataset, random_split
# import torch
import keras
import keras_nlp
import tensorflow as tf
import tensorflow_hub as hub
from tqdm import tqdm

In [2]:
data = pd.read_csv('Datasets/FIR_DATASET(updated).csv')
label_uni = data['section'].unique()
label_dict = {label_uni[i] : i for i in range(len(label_uni))}
data['label_id'] = data['section'].map(label_dict)
data['Description'] = data['Description'].astype(str)
data.head()

,URL,Description,Offense,Punishment,Cognizable,Bailable,Court,section,label_id
0,https://lawrato.com/indian-kanoon/ipc/section-140,Description of IPC Section 140\nAccording to s...,Wearing the dress or carrying any token used b...,3 Months or Fine or Both,Cognizable,Bailable,Any Magistrate,140,0
1,https://lawrato.com/indian-kanoon/ipc/section-127,Description of IPC Section 127\nAccording to s...,Receiving property taken by war or depredation...,7 Years + Fine + forfeiture of property,Cognizable,Non-Bailable,Court of Session,127,1
2,https://lawrato.com/indian-kanoon/ipc/section-128,Description of IPC Section 128\nAccording to s...,Public servant voluntarily allowing prisoner o...,Imprisonment for Life or 10 Years + Fine,Cognizable,Non-Bailable,Court of Session,128,2
3,https://lawrato.com/indian-kanoon/ipc/section-129,Description of IPC Section 129\nAccording to s...,Public servant negligently suffering prisoner ...,Simple Imprisonment 3 Years + Fine,Cognizable,Bailable,Magistrate First Class,129,3
4,https://lawrato.com/indian-kanoon/ipc/section-130,Description of IPC Section 130\nAccording to s...,"Aiding escape of, rescuing or harbouring, such...",Imprisonment for Life or 10 Years + Fine,Cognizable,Non-Bailable,Court of Session,130,4


In [3]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)


In [18]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(data['section'].unique()))
preproccessor = keras.models.load_model(
    'preprocessor_base.keras',
    custom_objects={'DistilBertTokenizer':keras_nlp.models.DistilBertClassifier}
)


TypeError: DistilBertClassifier.__init__() missing 2 required positional arguments: 'backbone' and 'num_classes'

In [7]:
# classifier.compile(
#     loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     optimizer = keras.optimizers.Adam(2e-3),
#     jit_compile = True,
#     metrics = ['accuracy'],
# )
# history = classifier.fit(
#     x = train_data['Description'],
#     y = train_data['label_id'],
#     batch_size = 32,
#     epochs = 200,
#     validation_data = (test_data['Description'], test_data['label_id']),
# )

2024-01-15 18:45:01.735191: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'bert_preprocessor/multi_segment_packer_1/Trimmer/Trim/RoundRobinTrimmer/generate_mask/while/Tile/multiples/bert_preprocessor/multi_segment_packer_1/Trimmer/Trim/RoundRobinTrimmer/generate_mask/strided_slice' with dtype int32
	 [[{{node bert_preprocessor/multi_segment_packer_1/Trimmer/Trim/RoundRobinTrimmer/generate_mask/while/Tile/multiples/bert_preprocessor/multi_segment_packer_1/Trimmer/Trim/RoundRobinTrimmer/generate_mask/strided_slice}}]]
2024-01-15 18:45:01.736781: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'bert_preprocessor/multi_seg

Epoch 1/200
12/12 [==============================] - ETA: 0s - loss: 6.4174 - accuracy: 0.0000e+00

2024-01-15 18:45:25.554849: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'bert_preprocessor/multi_segment_packer_1/Trimmer/Trim/RoundRobinTrimmer/generate_mask/while/Tile/multiples/bert_preprocessor/multi_segment_packer_1/Trimmer/Trim/RoundRobinTrimmer/generate_mask/strided_slice' with dtype int32
	 [[{{node bert_preprocessor/multi_segment_packer_1/Trimmer/Trim/RoundRobinTrimmer/generate_mask/while/Tile/multiples/bert_preprocessor/multi_segment_packer_1/Trimmer/Trim/RoundRobinTrimmer/generate_mask/strided_slice}}]]
2024-01-15 18:45:25.556668: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'bert_preprocessor/multi_seg

12/12 [==============================] - 26s 2s/step - loss: 6.4174 - accuracy: 0.0000e+00 - val_loss: 6.7341 - val_accuracy: 0.0000e+00
Epoch 2/200
12/12 [==============================] - ETA: 0s - loss: 6.0973 - accuracy: 0.0000e+00

2024-01-15 18:45:41.608580: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'bert_preprocessor/multi_segment_packer_1/Trimmer/Trim/RoundRobinTrimmer/generate_mask/while/Tile/multiples/bert_preprocessor/multi_segment_packer_1/Trimmer/Trim/RoundRobinTrimmer/generate_mask/strided_slice' with dtype int32
	 [[{{node bert_preprocessor/multi_segment_packer_1/Trimmer/Trim/RoundRobinTrimmer/generate_mask/while/Tile/multiples/bert_preprocessor/multi_segment_packer_1/Trimmer/Trim/RoundRobinTrimmer/generate_mask/strided_slice}}]]
2024-01-15 18:45:41.610222: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'bert_preprocessor/multi_seg

...

2024-01-15 18:46:40.507577: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'bert_preprocessor/multi_segment_packer_1/Trimmer/Trim/RoundRobinTrimmer/generate_mask/while/Tile/multiples/bert_preprocessor/multi_segment_packer_1/Trimmer/Trim/RoundRobinTrimmer/generate_mask/strided_slice' with dtype int32
	 [[{{node bert_preprocessor/multi_segment_packer_1/Trimmer/Trim/RoundRobinTrimmer/generate_mask/while/Tile/multiples/bert_preprocessor/multi_segment_packer_1/Trimmer/Trim/RoundRobinTrimmer/generate_mask/strided_slice}}]]
2024-01-15 18:46:40.509256: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'bert_preprocessor/multi_seg

12/12 [==============================] - 14s 1s/step - loss: 6.0099 - accuracy: 0.0000e+00 - val_loss: 10.3034 - val_accuracy: 0.0000e+00
Epoch 7/200
12/12 [==============================] - ETA: 0s - loss: 6.0275 - accuracy: 0.0056    

2024-01-15 18:46:55.279677: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'bert_preprocessor/multi_segment_packer_1/Trimmer/Trim/RoundRobinTrimmer/generate_mask/while/Tile/multiples/bert_preprocessor/multi_segment_packer_1/Trimmer/Trim/RoundRobinTrimmer/generate_mask/strided_slice' with dtype int32
	 [[{{node bert_preprocessor/multi_segment_packer_1/Trimmer/Trim/RoundRobinTrimmer/generate_mask/while/Tile/multiples/bert_preprocessor/multi_segment_packer_1/Trimmer/Trim/RoundRobinTrimmer/generate_mask/strided_slice}}]]
2024-01-15 18:46:55.281479: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'bert_preprocessor/multi_seg

12/12 [==============================] - 15s 1s/step - loss: 6.0275 - accuracy: 0.0056 - val_loss: 10.3443 - val_accuracy: 0.0000e+00
Epoch 8/200
 4/12 [=========>....................] - ETA: 9s - loss: 6.0534 - accuracy: 0.0000e+00 

KeyboardInterrupt: 

In [ ]:
# train_encodings = tokenizer(train_data['Description'].tolist(), padding=True, truncation=True, return_tensors='pt')
# train_labels = torch.tensor(train_data['section'].astype('category').cat.codes.tolist())

In [ ]:
# test_encodings = tokenizer(test_data['Description'].tolist(), padding=True, truncation=True, return_tensors='pt')
# test_labels = torch.tensor(test_data['section'].astype('category').cat.codes.tolist())

In [ ]:
# train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
# train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [ ]:
# test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)
# test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [ ]:
# optimizer = AdamW(model.parameters(), lr=2e-5)


In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)


In [ ]:
# for epoch in range(3):
#     model.train()
#     for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}'):
#         input_ids = batch[0].to(device)
#         attention_mask = batch[1].to(device)
#         labels = batch[2].to(device)

#         optimizer.zero_grad()
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()

In [ ]:
# model.eval()
# all_preds = []
# all_labels = []

In [ ]:
# with torch.no_grad():
#     for batch in tqdm(test_loader, desc='Testing'):
#         input_ids = batch[0].to(device)
#         attention_mask = batch[1].to(device)
#         labels = batch[2].to(device)

#         outputs = model(input_ids, attention_mask=attention_mask)
#         logits = outputs.logits
#         preds = torch.argmax(logits, dim=1).cpu().numpy()

#         all_preds.extend(preds)
#         all_labels.extend(labels.cpu().numpy())






In [ ]:
# accuracy = accuracy_score(all_labels, all_preds)
# print(f'Test Accuracy: {accuracy:.4f}')


In [ ]:
def suggest_section(complaint_description):
    tokenized_description = tokenizer(complaint_description, padding=True, truncation=True, return_tensors='pt')
    input_ids = tokenized_description['input_ids'].to(device)
    attention_mask = tokenized_description['attention_mask'].to(device)

    model.eval()
    with torch.no_grad():
        output = model(input_ids, attention_mask=attention_mask)
        logits = output.logits
        predicted_class = torch.argmax(logits, dim=1).item()

    # Convert the predicted class back to the original section label
    predicted_section = data['section'].unique()[predicted_class]
    return predicted_section


In [ ]:
# Example usage
complaint_description = "The suspect stole my wallet and assaulted me."
suggested_section = suggest_section(complaint_description)
print("Suggested Section:", suggested_section)


In [ ]:
def generate_fir():
    # Input from the user
    complainant_name = input("Enter complainant's name: ")
    father_name = input("Enter father's/husband's name: ")
    address = input("Enter address: ")
    phone_number = input("Enter phone number and fax: ")
    email = input("Enter email: ")
    place_of_occurrence = input("Enter place of occurrence: ")
    date_of_occurrence = input("Enter date and hour of occurrence: ")

    # Fetch suggestions from the model based on the complaint description
    complaint_description = input("Enter the complaint description: ")
    suggested_section = suggested_section(complaint_description, nlp, textcat.labels, data)

    # Retrieve additional information from the dataset based on the identified section
    section_info = data[data['section'] == suggested_section].iloc[0]
    bailable = section_info['Bailable']
    cognizable = section_info['Cognizable']
    court = section_info['Court']
    punishment = section_info['Punishment']
    offense_nature = section_info['Offense']  # Automatically fill the nature of the offense

    property_description = input("Enter particulars of the property: ")
    accused_description = input("Enter description of the accused: ")
    witness_details = input("Enter details of witnesses (if any): ")
    complaint = input("Enter complaint: ")

    # Create the FIR template
    fir_template = f"""Police Station: [Police Station]
District: [District]

1. Personal details of the Complainant / Informant:
(a) Name: {complainant_name}
(b) Father's / Husband's Name: {father_name}
(c) Address: {address}
(d) Phone number & Fax: {phone_number}
(e) Email: {email}

2. Place of Occurrence: {place_of_occurrence}

3. Date and Hour of Occurrence: {date_of_occurrence}

4. Offence:
(a) Nature of the offence: {offense_nature}  
(b) Section: {suggested_section}  
(c) Particulars of the property: {property_description}

5. Description of the accused: {accused_description} 

6. Additional Section Information:
   - Bailable: {bailable}
   - Cognizable: {cognizable}
   - Court: {court}
   - Punishment: {punishment}

7. Details of witnesses (if any): {witness_details}

8. Complaint: {complaint}
"""

    # Return the generated FIR
    return fir_template

In [ ]:
fir_text = generate_fir()
print(fir_text)